In [ ]:
!!pip install litellm

# Important!!!
#
# <---- Set your 'OPENAI_API_KEY' as a secret over there with the "key" icon
#
#
import os
from google.colab import userdata
api_key = userdata.get('OPENAI_API_KEY')
os.environ['OPENAI_API_KEY'] = api_key

In [ ]:
import json
import os
from typing import List

from litellm import completion

def list_files() -> List[str]:
    """List files in the current directory."""
    return os.listdir(".")

def read_file(file_name: str) -> str:
    """Read a file's contents."""
    try:
        with open(file_name, "r") as file:
            return file.read()
    except FileNotFoundError:
        return f"Error: {file_name} not found."
    except Exception as e:
        return f"Error: {str(e)}"

def terminate(message: str) -> None:
    """Terminate the agent loop and provide a summary message."""
    print(f"Termination message: {message}")

tool_functions = {
    "list_files": list_files,
    "read_file": read_file,
    "terminate": terminate
}

tools = [
    {
        "type": "function",
        "function": {
            "name": "list_files",
            "description": "Returns a list of files in the directory.",
            "parameters": {"type": "object", "properties": {}, "required": []}
        }
    },
    {
        "type": "function",
        "function": {
            "name": "read_file",
            "description": "Reads the content of a specified file in the directory.",
            "parameters": {
                "type": "object",
                "properties": {"file_name": {"type": "string"}},
                "required": ["file_name"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "terminate",
            "description": "Terminates the conversation. No further actions or interactions are possible after this. Prints the provided message for the user.",
            "parameters": {
                "type": "object",
                "properties": {
                    "message": {"type": "string"},
                },
                "required": ["message"]
            }
        }
    }
]

agent_rules = [{
    "role": "system",
    "content": """
You are an AI agent that can perform tasks by using available tools.

If a user asks about files, documents, or content, first list the files before reading them.

When you are done, terminate the conversation by using the "terminate" tool and I will provide the results to the user.
"""
}]

# Initialize agent parameters
iterations = 0
max_iterations = 10

user_task = input("What would you like me to do? ")

memory = [{"role": "user", "content": user_task}]

# The Agent Loop
while iterations < max_iterations:

    messages = agent_rules + memory

    response = completion(
        model="openai/gpt-4o",
        messages=messages,
        tools=tools,
        max_tokens=1024
    )

    if response.choices[0].message.tool_calls:
        tool = response.choices[0].message.tool_calls[0]
        tool_name = tool.function.name
        tool_args = json.loads(tool.function.arguments)

        action = {
            "tool_name": tool_name,
            "args": tool_args
        }

        if tool_name == "terminate":
            print(f"Termination message: {tool_args['message']}")
            break
        elif tool_name in tool_functions:
            try:
                result = {"result": tool_functions[tool_name](**tool_args)}
            except Exception as e:
                result = {"error":f"Error executing {tool_name}: {str(e)}"}
        else:
            result = {"error": f"Unknown tool: {tool_name}"}

        print(f"Executing: {tool_name} with args {tool_args}")
        print(f"Result: {result}")
        memory.extend([
            {"role": "assistant", "content": json.dumps(action)},
            {"role": "user", "content": json.dumps(result)}
        ])
    else:
        result = response.choices[0].message.content
        print(f"Response: {result}")
        break


What would you like me to do? read each of the files in the current directory and tell me what they do
Executing: list_files with args {}
Result: {'result': ['.config', 'sample_data']}
Executing: read_file with args {'file_name': '.config'}
Result: {'result': "Error: [Errno 21] Is a directory: '.config'"}
Termination message: It seems that ".config" is a directory, not a file. If you want specific information from within it, please let me know, or ensure there are files inside it to read.


Setting Up Your Simulation
When starting a conversation with an LLM to simulate your agent, begin by establishing the framework. We can do this with a simple prompt in a chat interface. The prompt should clearly outline the agent’s goals, actions, and the simulation process. Here’s a template you can use:

I'd like to simulate an AI agent that I'm designing. The agent will be built using these components:

Goals: [List your goals]
Actions: [List available actions]

At each step, your output must be an action to take.

Stop and wait and I will type in the result of
the action as my next message.

Ask me for the first task to perform.
For a Proactive Coder agent, you might use the following prompt to kick-off a simulation in ChatGPT:

I'd like to simulate an AI agent that I'm designing. The agent will be built using these components:

Goals:
* Find potential code enhancements
* Ensure changes are small and self-contained
* Get user approval before making changes
* Maintain existing interfaces

Actions available:
* list_project_files()
* read_project_file(filename)
* ask_user_approval(proposal)
* edit_project_file(filename, changes)

At each step, your output must be an action to take.

Stop and wait and I will type in the result of
the action as my next message.

Ask me for the first task to perform.
Take a moment to open up ChatGPT and try out this prompt. You can use the same prompt in any chat interface that supports LLMs. What worked? What didn’t?

Learning Through Agent Simulation
Understanding Agent Reasoning
When you begin simulating your agent’s behavior, you’re essentially conducting a series of experiments to understand how well it can reason with the tools and goals you’ve provided. Start by presenting a simple scenario – perhaps a small Python project with just a few files. Watch how the agent approaches the task. Does it immediately jump to reading files, or does it first list the available files to get an overview? These initial decisions reveal a lot about whether your goals and actions enable systematic problem-solving.

As you observe the agent’s decisions, you’ll notice that the way you present information significantly impacts its reasoning. For instance, when you return the results of list_project_files(), you might first try returning just the filenames:

["main.py", "utils.py", "data_processor.py"]
Then experiment with providing more context:

{
    "files": ["main.py", "utils.py", "data_processor.py"],
    "total_files": 3,
    "directory": "/project"
}
You might discover that the additional metadata helps the agent make more informed decisions about which files to examine next. This kind of experimentation with result formats helps you understand how much context your agent needs to reason effectively.

Evolving Your Tools and Goals
The simulation process often reveals that your initial tool descriptions aren’t as clear as you thought. For example, you might start with a simple description for read_project_file():

read_project_file(filename) -> Returns the content of the specified file
Through simulation, you might find the agent using it incorrectly, leading you to enhance the description:

read_project_file(filename) -> Returns the content of a Python file from the project directory.
The filename should be one previously returned by list_project_files().
Similarly, your goals might evolve. You might start with “Find potential code enhancements” but discover through simulation that the agent needs more specific guidance. This might lead you to refine the goal to “Identify opportunities to improve error handling and input validation in functions.”

Understanding Memory Through Chat
One of the most enlightening aspects of simulation is realizing that the chat format naturally mimics the list-based memory system we use in our agent loop memory. Each exchange between you and the LLM represents an iteration of the agent loop and a new memory entry – the agent’s actions and the environment’s responses accumulate just as they would in our implemented memory system. This helps you understand how much history the agent can accumulate and still maintain context and make good decisions.

Learning from Failures
Introducing controlled chaos into your simulation provides valuable insights. Try returning error messages instead of successful results:

{"error": "FileNotFoundError: main.py does not exist"}
Or return malformed data:

{"cont3nt": "def broken_func(): pass"}
Watch how the agent handles these situations. Does it try alternative approaches? Does it give up too easily? Does it maintain its goal focus despite errors? These observations help you design better error handling and recovery strategies.

Preventing Runaway Agents
The simulation environment provides a safe space to test termination conditions. You can experiment with different criteria for when the agent should conclude its task. Perhaps it should stop after examining a certain number of files, or after making a specific number of improvement suggestions. The chat format lets you quickly try different approaches without worrying about infinite loops or resource consumption.

Rapid Iteration and Improvement
The true power of simulation lies in its speed. You can test dozens of scenarios in the time it would take to implement a single feature. Want to see how the agent handles a project with 100 files? Just tell it that’s what list_project_files() returned. Curious about how it would handle deeply nested function calls? Paste in some complex code and see how it analyzes it.

Learning from the Agent
At the end of your simulation sessions, ask the agent to reflect on its experience. What tools did it wish it had? Were any instructions unclear? Which goals were too vague? The LLM can often provide surprisingly insightful suggestions about how to improve your GAME design.

For example, the agent might suggest: “The ask_user_approval() action would be more effective if it could include code snippets showing the proposed changes. This would help users make more informed decisions about the suggested improvements.”

Building Your Example Library
As you conduct these simulations, you’re building a valuable library of examples. When you see the agent make a particularly good decision, save that exchange. When it makes a poor choice, save that too. These examples become invaluable when you move to implementation – they can be used to craft better prompts and test cases.

Keep a record of exchanges like this:

Good Example:

Agent: "Before modifying utils.py, I should read its contents to understand the current error handling patterns."
Action: read_project_file("utils.py")
Result: [file contents]
Agent: "I notice these functions lack input validation. I'll propose focused improvements for each function."
Poor Example:

Agent: "I'll start editing all the files to add error handling."
Action: edit_project_file("utils.py", {...})
[Missing analysis and user approval steps]
These examples help you understand what patterns to encourage or discourage in your implemented agent.

Through this iterative process of simulation, observation, and refinement, you develop a deep understanding of how your agent will behave in the real world. This understanding is invaluable when you move to implementation, helping you build agents that are more robust, more capable, and better aligned with your goals.

Remember, the time spent in simulation is an investment that pays off in better design decisions and fewer implementation surprises. When you finally start coding, you’re not just hoping your design will work – you’ve already seen it work in hundreds of scenarios.

Building a Simple Agent Framework 1
We are designing our agents in terms of GAME. Ideally, we would like our code to reflect how we design the agent, so that we can easily translate our design into an implementation. Also, we can see that the GAME components are what change from one agent to another while the core loop stays the same. We would like to design a framework that allows us to reuse as much as possible while making it easy to change the GAME pieces without affecting the GAME rules (e.g., the agent loop).

At first, it will appear that we are adding complexity to the agent — and we are. However, this complexity is necessary to create a framework that is flexible and reusable. The goal is to create a framework that allows us to build agents quickly and easily without changing the core loop. We are going to look at each of the individual GAME component implementations and then how they fit into the overall framework at the end.

G - Goals Implementation
First, let’s create a simple goal class that defines what our agent is trying to accomplish:

@dataclass(frozen=True)
class Goal:
    priority: int
    name: str
    description: str
Goals will describe what we are trying to achieve and how to achieve it. By encapsulating them into objects, we can move away from large “walls of text” that represent the instructions for our agent. Additionally, we can add priority to our goals, which will help us decide which goal to pursue first and how to sort or format them when combining them into a prompt.

We broadly use the term “goal” to encompass both “what” the agent is trying to achieve and “how” it should approach the task. This duality is crucial for guiding the agent’s behavior effectively. An important type of goal can be examples that show the agent how to reason in certain situations. We can also build goals that define core rules that are common across all agents in our system or that give it special instructions on how to solve certain types of tasks.

Now, let’s take a look at how we might create a goal related to file management for our agent:

from game.core import Goal

# Define a simple file management goal
file_management_goal = Goal(
    priority=1,
    name="file_management",
    description="""Manage files in the current directory by:
    1. Listing files when needed
    2. Reading file contents when needed
    3. Searching within files when information is required
    4. Providing helpful explanations about file contents"""
)
A - Actions Implementation with JSON Schemas
Actions define what the agent can do. Think of them as the agent’s toolkit. Each action is a discrete capability that can be executed in the environment. The action system has two main parts: the Action class and the ActionRegistry.

The actions are the interface between our agent and its environment. These are descriptions of what the agent can do to affect the environment. We have previously built out actions using Python functions, but let’s encapsulate the parts of an action into an object:

class Action:
    def __init__(self,
                 name: str,
                 function: Callable,
                 description: str,
                 parameters: Dict,
                 terminal: bool = False):
        self.name = name
        self.function = function
        self.description = description
        self.terminal = terminal
        self.parameters = parameters

    def execute(self, **args) -> Any:
        """Execute the action's function"""
        return self.function(**args)
At first, it may not appear that this is much different from the previous implementation. However, later, we will see that this makes it much easier to create different agents by simply swapping out the actions without having to modify the core loop.

When the agent provides a response, it is going to return JSON. However, we are going to want a way to lookup the actual object associated with the action indicated by the JSON. To do this, we will create an ActionRegistry that will allow us to register actions and look them up by name:

class ActionRegistry:
    def __init__(self):
        self.actions = {}

    def register(self, action: Action):
        self.actions[action.name] = action

    def get_action(self, name: str) -> [Action, None]:
        return self.actions.get(name, None)

    def get_actions(self) -> List[Action]:
        """Get all registered actions"""
        return list(self.actions.values())
Here is an example of how we might define some actions for a file management agent:

def list_files() -> list:
    """List all files in the current directory."""
    return os.listdir('.')

def read_file(file_name: str) -> str:
    """Read and return the contents of a file."""
    with open(file_name, 'r') as f:
        return f.read()

def search_in_file(file_name: str, search_term: str) -> list:
    """Search for a term in a file and return matching lines."""
    results = []
    with open(file_name, 'r') as f:
        for i, line in enumerate(f.readlines()):
            if search_term in line:
                results.append((i+1, line.strip()))
    return results

# Create and populate the action registry
registry = ActionRegistry()

registry.register(Action(
    name="list_files",
    function=list_files,
    description="List all files in the current directory",
    parameters={
        "type": "object",
        "properties": {},
        "required": []
    },
    terminal=False
))

registry.register(Action(
    name="read_file",
    function=read_file,
    description="Read the contents of a specific file",
    parameters={
        "type": "object",
        "properties": {
            "file_name": {
                "type": "string",
                "description": "Name of the file to read"
            }
        },
        "required": ["file_name"]
    },
    terminal=False
))

registry.register(Action(
    name="search_in_file",
    function=search_in_file,
    description="Search for a term in a specific file",
    parameters={
        "type": "object",
        "properties": {
            "file_name": {
                "type": "string",
                "description": "Name of the file to search in"
            },
            "search_term": {
                "type": "string",
                "description": "Term to search for"
            }
        },
        "required": ["file_name", "search_term"]
    },
    terminal=False
))
M - Memory Implementation
Almost every agent needs to remember what happens from one loop iteration to the next. This is where the Memory component comes in. It allows the agent to store and retrieve information about its interactions, which is critical for context and decision-making. We can create a simple class to represent the memory:

class Memory:
    def __init__(self):
        self.items = []  # Basic conversation histor

    def add_memory(self, memory: dict):
        """Add memory to working memory"""
        self.items.append(memory)

    def get_memories(self, limit: int = None) -> List[Dict]:
        """Get formatted conversation history for prompt"""
        return self.items[:limit]
Originally, we just used a simple list of messages. Is it worth wrapping the list in this additional class? Yes, because it allows us to add additional functionality later without changing the core loop. For example, we might want to store the memory in a database and dynamically change what memories the agent sees at each loop iteration based on some analysis of the state of the memory. With this simple interface, we can create subclasses that implement different memory strategies without changing the core loop.

One thing to note is that our memory always has to be represented as a list of messages in the prompt. Because of this, we provide a simple interface to the memory that returns the last N messages in the correct format. This allows us to keep the memory class agnostic to how it is used. We can change how we store the memory (e.g., in a database) without changing how we access it in the agent loop. Even if we store the memory in a complicated graph structure, we are still going to need to pass the memories to the LLM as a list and format them as messages.

E - Environment Implementation
In our original implementation, we hardcoded our “environment” interface as a series of if/else statements and function calls. We would like to have a more modular interface that allows us to execute actions without needing to know how they are implemented or have conditional logic in the loop. This is where the Environment component comes in. It serves as a bridge between the agent and the outside world, executing actions and returning results.

class Environment:
    def execute_action(self, action: Action, args: dict) -> dict:
        """Execute an action and return the result."""
        try:
            result = action.execute(**args)
            return self.format_result(result)
        except Exception as e:
            return {
                "tool_executed": False,
                "error": str(e),
                "traceback": traceback.format_exc()
            }

    def format_result(self, result: Any) -> dict:
        """Format the result with metadata."""
        return {
            "tool_executed": True,
            "result": result,
            "timestamp": time.strftime("%Y-%m-%dT%H:%M:%S%z")
        }

Building a Simple Agent Framework 2
Now, we are going to put the components together into a reusable agent class. This class will encapsulate the GAME components and provide a simple interface for running the agent loop. The agent will be responsible for constructing prompts, executing actions, and managing memory. We can create different agents simply by changing the goals, actions, and environment without modifying the core loop.

Let’s take a look at our agent class:

class Agent:
    def __init__(self,
                 goals: List[Goal],
                 agent_language: AgentLanguage,
                 action_registry: ActionRegistry,
                 generate_response: Callable[[Prompt], str],
                 environment: Environment):
        """
        Initialize an agent with its core GAME components
        """
        self.goals = goals
        self.generate_response = generate_response
        self.agent_language = agent_language
        self.actions = action_registry
        self.environment = environment

    def construct_prompt(self, goals: List[Goal], memory: Memory, actions: ActionRegistry) -> Prompt:
        """Build prompt with memory context"""
        return self.agent_language.construct_prompt(
            actions=actions.get_actions(),
            environment=self.environment,
            goals=goals,
            memory=memory
        )

    def get_action(self, response):
        invocation = self.agent_language.parse_response(response)
        action = self.actions.get_action(invocation["tool"])
        return action, invocation

    def should_terminate(self, response: str) -> bool:
        action_def, _ = self.get_action(response)
        return action_def.terminal

    def set_current_task(self, memory: Memory, task: str):
        memory.add_memory({"type": "user", "content": task})

    def update_memory(self, memory: Memory, response: str, result: dict):
        """
        Update memory with the agent's decision and the environment's response.
        """
        new_memories = [
            {"type": "assistant", "content": response},
            {"type": "user", "content": json.dumps(result)}
        ]
        for m in new_memories:
            memory.add_memory(m)

    def prompt_llm_for_action(self, full_prompt: Prompt) -> str:
        response = self.generate_response(full_prompt)
        return response

    def run(self, user_input: str, memory=None, max_iterations: int = 50) -> Memory:
        """
        Execute the GAME loop for this agent with a maximum iteration limit.
        """
        memory = memory or Memory()
        self.set_current_task(memory, user_input)

        for _ in range(max_iterations):
            # Construct a prompt that includes the Goals, Actions, and the current Memory
            prompt = self.construct_prompt(self.goals, memory, self.actions)

            print("Agent thinking...")
            # Generate a response from the agent
            response = self.prompt_llm_for_action(prompt)
            print(f"Agent Decision: {response}")

            # Determine which action the agent wants to execute
            action, invocation = self.get_action(response)

            # Execute the action in the environment
            result = self.environment.execute_action(action, invocation["args"])
            print(f"Action Result: {result}")

            # Update the agent's memory with information about what happened
            self.update_memory(memory, response, result)

            # Check if the agent has decided to terminate
            if self.should_terminate(response):
                break

        return memory
Now, let’s walk through how the GAME components work together in this agent architecture, explaining each part of agent loop.

Step 1: Constructing the Prompt
When the agent loop begins, it first constructs a prompt using the construct_prompt method:

def construct_prompt(self, goals: List[Goal], memory: Memory, actions: ActionRegistry) -> Prompt:
    """Build prompt with memory context"""
    return self.agent_language.construct_prompt(
        actions=actions.get_actions(),
        environment=self.environment,
        goals=goals,
        memory=memory
    )
This method leverages the AgentLanguage component to build a structured prompt containing:

The agent’s goals (what it’s trying to accomplish)
Available actions (tools the agent can use)
Current memory context (conversation history and relevant information)
Environment details (constraints and context for operation)
We are going to discuss the AgentLanguage in more detail later. For now, what you need to know is that it is responsible for formatting the prompt that is sent to the LLM and parsing the response from the LLM. Most of the time, we are going to use function calling, so the parsing will just be reading the returned tool calls. However, the AgentLanguage can be changed to allow us to also take the same agent and implement it without function calling.

Step 2: Generating a Response
Next, the agent sends this prompt to the language model:

def prompt_llm_for_action(self, full_prompt: Prompt) -> str:
    response = self.generate_response(full_prompt)
    return response
The generate_response function is a simple python function provided during initialization. This abstraction allows the framework to work with different language models without changing the core loop. We will use LiteLLM to call the LLM, but you could easily swap this out for any other LLM provider.

Step 3: Parsing the Response
Once the language model returns a response, the agent parses it to identify the intended action. The parsing will generally be just getting the tool calls from the response, however the agent language gets to decide how this is done. Once the response is parsed, the agent can look up the action in the ActionRegistry:

def get_action(self, response):
    invocation = self.agent_language.parse_response(response)
    action = self.actions.get_action(invocation["tool"])
    return action, invocation
The action is the interface definition of what the agent “can” do. The invocation is the specific parameters that the agent has chosen to use for this action. The ActionRegistry allows the agent to look up the action by name, and the invocation provides the arguments needed to execute it. We could also add validation at this step to ensure that the invocation parameters match the action’s expected parameters.

Step 4: Executing the Action
The agent then executes the chosen action in the environment:

# Execute the action in the environment
result = self.environment.execute_action(action, invocation["args"])
The Environment handles the actual execution of the action, which might involve:

Making API calls
Reading/writing files
Querying databases
Processing data
Actions are defined in the ActionRegistry but executed within the context of the Environment, which provides access to resources and handles the mechanics of execution.

Step 5: Updating Memory
After execution, the agent updates its memory with both its decision and the result:

def update_memory(self, memory: Memory, response: str, result: dict):
    """
    Update memory with the agent's decision and the environment's response.
    """
    new_memories = [
        {"type": "assistant", "content": response},
        {"type": "user", "content": json.dumps(result)}
    ]
    for m in new_memories:
        memory.add_memory(m)
This creates a continuous record of the agent’s reasoning and actions, which becomes part of the context of future loop iterations. The memory serves both as a record of past actions and as context for future prompt construction.

Step 6: Termination Check
Finally, the agent checks if it should terminate the loop:

def should_terminate(self, response: str) -> bool:
    action_def, _ = self.get_action(response)
    return action_def.terminal
This allows certain actions (like a “terminate” action) to signal that the agent has finished its work.

The Flow of Information Through the Loop
To better understand how these components interact, let’s trace how information flows through a single iteration of the loop:

The Memory provides context about what the user has asked the agent to do and past decisions and results from the agent loop
The Goals define what the agent is trying to accomplish and rules on how to accomplish it
The ActionRegistry defines what the agent can do and helps lookup the action to execute by name
The AgentLanguage formats Memory, Actions, and Goals into a prompt for the LLM
The LLM generates a response choosing an action
The AgentLanguage parses the response into an action invocation, which will typically be extracted from tool calls
The Environment executes the action with the given arguments
The result is stored back in Memory
The loop repeats with the updated memory until the agent calls a terminal tool or reaches the maximum number of iterations
Creating Specialized Agents
The beauty of this framework is that we can create entirely different agents by changing the GAME components without modifying the core loop:

# A research agent
research_agent = Agent(
    goals=[Goal("Find and summarize information on topic X")],
    agent_language=ResearchLanguage(),
    action_registry=ActionRegistry([SearchAction(), SummarizeAction(), ...]),
    generate_response=openai_call,
    environment=WebEnvironment()
)

# A coding agent
coding_agent = Agent(
    goals=[Goal("Write and debug Python code for task Y")],
    agent_language=CodingLanguage(),
    action_registry=ActionRegistry([WriteCodeAction(), TestCodeAction(), ...]),
    generate_response=anthropic_call,
    environment=DevEnvironment()
)
Each agent operates using the same fundamental loop but exhibits completely different behaviors based on its GAME components.

Building a Simple Agent Framework 3
Let’s go back to the file agent that we built earlier. The original implementation uses direct function calls and a lot of conditional logic in the agent loop. Let’s redo the implementation using our new framework.

Define the Goals
First, let’s define some goals for our file explorer agent:

# Define clear goals for the agent
goals = [
    Goal(
        priority=1,
        name="Explore Files",
        description="Explore files in the current directory by listing and reading them"
    ),
    Goal(
        priority=2,
        name="Terminate",
        description="Terminate the session when tasks are complete with a helpful summary"
    )
]
Create Actions Using the Framework
Next, let’s convert our tool functions into properly structured Actions in our AgentRegistry:

def list_files() -> List[str]:
    """List files in the current directory."""
    return os.listdir(".")

def read_file(file_name: str) -> str:
    """Read a file's contents."""
    try:
        with open(file_name, "r") as file:
            return file.read()
    except FileNotFoundError:
        return f"Error: {file_name} not found."
    except Exception as e:
        return f"Error: {str(e)}"

def terminate(message: str) -> str:
    """Terminate the agent loop and provide a summary message."""
    return message

# Create and register the actions
action_registry = ActionRegistry()

action_registry.register(Action(
    name="list_files",
    function=list_files,
    description="Returns a list of files in the directory.",
    parameters={},
    terminal=False
))

action_registry.register(Action(
    name="read_file",
    function=read_file,
    description="Reads the content of a specified file in the directory.",
    parameters={
        "type": "object",
        "properties": {
            "file_name": {"type": "string"}
        },
        "required": ["file_name"]
    },
    terminal=False
))

action_registry.register(Action(
    name="terminate",
    function=terminate,
    description="Terminates the conversation. Prints the provided message for the user.",
    parameters={
        "type": "object",
        "properties": {
            "message": {"type": "string"},
        },
        "required": ["message"]
    },
    terminal=True
))
Create and Run the Agent
Now we can put it all together:


# Create the agent
file_explorer_agent = Agent(
    goals=goals,
    agent_language=agent_language,
    action_registry=action_registry,
    generate_response=generate_response,
    environment=environment
)

# Run the agent
user_input = input("What would you like me to do? ")
final_memory = file_explorer_agent.run(user_input, max_iterations=10)

# Print the final conversation if desired
for item in final_memory.get_memories():
    print(f"\n{item['type'].upper()}: {item['content']}")
Complete Implementation
Here’s the full implementation using the GAME framework:

def main():
    # Define the agent's goals
    goals = [
        Goal(
            priority=1,
            name="Explore Files",
            description="Explore files in the current directory by listing and reading them"
        ),
        Goal(
            priority=2,
            name="Terminate",
            description="Terminate the session when tasks are complete with a helpful summary"
        )
    ]
    
    # Define tool functions
    def list_files() -> List[str]:
        """List files in the current directory."""
        return os.listdir(".")

    def read_file(file_name: str) -> str:
        """Read a file's contents."""
        try:
            with open(file_name, "r") as file:
                return file.read()
        except FileNotFoundError:
            return f"Error: {file_name} not found."
        except Exception as e:
            return f"Error: {str(e)}"

    def terminate(message: str) -> str:
        """Terminate the agent loop and provide a summary message."""
        return message
    
    # Create action registry and register actions
    action_registry = ActionRegistry()
    
    action_registry.register(Action(
        name="list_files",
        function=list_files,
        description="Returns a list of files in the directory.",
        parameters={},
        terminal=False
    ))
    
    action_registry.register(Action(
        name="read_file",
        function=read_file,
        description="Reads the content of a specified file in the directory.",
        parameters={
            "type": "object",
            "properties": {
                "file_name": {"type": "string"}
            },
            "required": ["file_name"]
        },
        terminal=False
    ))
    
    action_registry.register(Action(
        name="terminate",
        function=terminate,
        description="Terminates the conversation. Prints the provided message for the user.",
        parameters={
            "type": "object",
            "properties": {
                "message": {"type": "string"},
            },
            "required": ["message"]
        },
        terminal=True
    ))
    
    # Define the agent language and environment
    agent_language = AgentFunctionCallingActionLanguage()
    environment = Environment()
    
    # Create the agent
    file_explorer_agent = Agent(
        goals=goals,
        agent_language=agent_language,
        action_registry=action_registry,
        generate_response=generate_response,
        environment=environment
    )
    
    # Run the agent
    user_input = input("What would you like me to do? ")
    final_memory = file_explorer_agent.run(user_input, max_iterations=10)
    
    # Print the termination message (if any)
    for item in final_memory.get_memories():
        print(f"\nMemory: {item['content']}")

if __name__ == "__main__":
    main()
Key Differences and Benefits
By converting our agent to the GAME framework, we gain several benefits:

Better Organization: Each component has a clear purpose and is separated from others.
Reusability: We can swap out components (like the actions or environment) without changing the core logic.
Extensibility: New goals and actions can be added easily.
Standard Interface: Using the Agent class gives us a consistent way to interact with different agents.
Memory Management: The framework handles memory updates automatically.
This structure also makes it easier to understand and maintain the code, especially as your agent grows in complexity.

Using the Agent
Once implemented, you can use your file explorer agent like this:

What would you like me to do? Tell me what Python files are in this directory and summarize how they fit together.

Agent thinking...
Agent Decision: I'll help you explore the Python files in this directory.

{"tool_name": "list_files", "args": {}}

Action Result: {'tool_executed': True, 'result': ['file1.py', 'file2.py', 'main.py', ...], 'timestamp': '2025-03-02T12:34:56+0000'}

{"tool_name": "read_file", "args": {"file_name": "file1.py"}}

Action Result: {'tool_executed': True, 'result': '# This is file1.py\n\ndef hello_world():\n    print("Hello, World!")\n\nif __name__ == "__main__":\n    hello_world()', 'timestamp': '2025-03-02T12:34:58+0000'}

[Additional file readings...]

{"tool_name": "terminate", "args": {"message": "I've explored all Python files in this directory. Here's a summary: file1.py contains a simple hello_world function, file2.py implements a calculator class, and main.py imports both files and uses their functionality."}}

...
This structured approach makes it much easier to develop, maintain, and extend your agents over time.